In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib import algos, helpers, envs
from lib.helpers import np_exclude
import numpy as np
import pandas as pd
import itertools

import matplotlib.pyplot as plt
import matplotlib.patches as patches

import copy
from tqdm import tqdm

import matplotlib
%config InlineBackend.figure_formats = ['svg']

In [3]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"],
})

In [4]:
np.set_printoptions(precision=5, suppress=True)

In [5]:
n_ep = 5000
epsilon = 0.9
# alpha_max, alpha_min, alpha_decay = 0.9, 1e-15, 1e-3
# alpha = lambda ep: alpha_min + (alpha_max-alpha_min)*np.exp(-alpha_decay*(ep))
# alpha = 0.1
gamma = 0.9

## Algorithms

### LSPI

In [6]:
# Model-based least-squares policy iteration
# using linear least-squares fixed-point approximation
# P: (nS*nA, dSA)
# R: (nS*nA,)
# X: (nS*nA, nS)
def LSPI_impl(P, R, X, γ, w_init=None, max_iter=100):
    _, dSA = X.shape
    nS, nA = env.nS, env.nA
    
    w_history = []

    if w_init is not None:
        w = w_init.copy()
    else:
        w = np.zeros((dSA,))

    w_history.append(w)
    
    for k in range(max_iter):
        π_w = (X @ w).reshape((nS, nA)).argmax(axis=1)

        Pi_w = np.zeros((nS, nS, nA))
        for s in range(nS):
            Pi_w[s,s,π_w[s]] = 1

        Pi_w = Pi_w.reshape((nS, -1))

        A = X.T @ (X - γ * P @ Pi_w @ X)
        b = X.T @ R
        w_ = np.linalg.pinv(A) @ b

        if np.isclose(w, w_).all():
            w_history.append(w_)
            break

        w = w_
        w_history.append(w)
    
    return w, w_history

In [7]:
# LSPI using one-hot state-action featurization
def LSPI_onehot(
    env, γ,
    w_init=None,
    max_iter=100,
):
    P, R, X = env.featurize(factored=False)
    return LSPI_impl(P, R, X, γ, w_init=w_init, max_iter=max_iter)

In [8]:
# LSPI using factored-action featurization (state is still equivalent of one-hot)
def LSPI_factored(
    env, γ,
    w_init=None,
    max_iter=100,
):
    P, R, X = env.featurize(factored=True)
    return LSPI_impl(P, R, X, γ, w_init=w_init, max_iter=max_iter)

### FactoredQ

In [9]:
class FactoredQ(object):
    def __init__(self, nSs, nAs):
        self.nSs = nSs
        self.nAs = nAs
        self.nS = nS = np.product(nSs)
        self.nA = nA = np.product(nAs)
        self.V0 = np.zeros((self.nS))
        self.Aj = [np.zeros((self.nS, nAj)) for nAj in self.nAs] # first column of each Aj[j] should always be zero
    
    def __getitem__(self, key):
        if isinstance(key, tuple):
            s, a = key
            return self.V0[s] + sum(self.Aj[j][s, a[j]] for j in range(len(self.nAs)))
        else: # assume only s is provided
            s = key
            Aj_all = np.zeros(self.nAs)
            for j in range(len(self.nAs)):
                Aj_all += np.expand_dims(self.Aj[j][s], axis=tuple([i for i in range(len(self.nAs)) if i != j]))
            return self.V0[s] + Aj_all
    
    def update(self, key, value, alpha):
        s, a = key
        Q_target = value
        
        # compute values
        new_V0s = Q_target - sum(self.Aj[j][s, a[j]] for j in range(len(self.nAs)))
        Ajsaj = np.zeros(len(self.nAs))
        for j in range(len(self.nAs)):
            if a[j] == 0: continue
            Ajsaj[j] = Q_target - self.V0[s] - sum(self.Aj[i][s, a[i]] for i in range(len(self.nAs)) if i != j)
        
        # update values
        self.V0[s] = (1-alpha) * self.V0[s] + alpha * new_V0s
        for j in range(len(self.nAs)):
            if a[j] == 0: continue
            self.Aj[j][s, a[j]] = (1-alpha) * self.Aj[j][s, a[j]] + alpha * Ajsaj[j]


def qlearn_factored(
    env, n_episodes, behavior_policy, gamma, alpha=0.1, epsilon=1.0, 
    Q_init=None, memory=None, save_Q=0, use_tqdm=True,
):
    if not callable(epsilon):
        epsilon_func = lambda episode: epsilon
    else:
        epsilon_func = epsilon
    
    if not callable(alpha): # step size
        alpha_ = alpha
        alpha = lambda episode: alpha_
    
    Q = FactoredQ(env.nSs, env.nAs)
    if Q_init is not None:
        Q = copy.deepcopy(Q_init)
    
    Gs = []
    Qs = [copy.deepcopy(Q)]
    TD_errors = []
    memory_buffer = []
    
    for episode in tqdm(range(n_episodes), disable=(not use_tqdm)):
        G = 0
        t = 0
        np.random.seed(episode)
        env.seed(episode)
        env.reset()
        S = env.s
        done = False
        while not done: # S is not a terminal state
            p = behavior_policy(np.reshape(Q[S],(1,-1)), dict(epsilon=epsilon_func(episode)))[0]
            A = np.random.choice(env.nA, p=p)
            
            AA = env.encode_action(A) # A // 2, A % 2
            
            S_, R, done, info = env.step(A)
            memory_buffer.append((S, A, R, S_, done))
            TD_errors.append(R + gamma * Q[S_].max() - Q[S,AA])
            
            # Perform update
            ##Q_new = Q[S,AA] + alpha(episode) * (R + gamma * Q[S_].max() - Q[S,AA])
            ##Q[S,AA] = Q_new
            
            Q_target = R + gamma * Q[S_].max()
            Q.update((S, AA), Q_target, alpha(episode))
            
            S = S_
            G = G + (gamma ** t) * R
            t = t + 1
            if save_Q: Qs.append(copy.deepcopy(Q))
        Gs.append(G)
    
    return Q, {
        'Gs': np.array(Gs), # cumulative reward for each episode
        'Qs': Qs, # history of all Q-values per update
        'TD_errors': np.array(TD_errors), # temporal difference error for each update
        'memory': memory_buffer, # all trajectories/experience, tuples of (s,a,r,s', done)
    }


## Environments

### Base MDP class

In [10]:
import gym
from gym.envs.toy_text import discrete
import numpy as np

class MDP_Env(discrete.DiscreteEnv):
    def __init__(self, p_transition, p_reward, S_terminal, isd=None):
        nS, nA, _ = p_transition.shape
        if isd is None:
            isd = np.ones(nS)
            isd = isd / isd.sum()
        
        if len(p_reward.shape) == 1:
            p_reward = np.tile(p_reward, (nS, nA, 1))
        if len(p_reward.shape) == 2:
            p_reward = np.repeat(p_reward, nS).reshape((nS, nA, nS))
        
        P = {s : {a : [] for a in range(nA)} for s in range(nS)}
        for s in range(nS):
            for a in range(nA):
                for s_ in range(nS):
                    if p_transition[s,a,s_] > 0:
                        P[s][a].append((p_transition[s,a,s_], s_, p_reward[s,a,s_], s_ in S_terminal))
        super().__init__(nS, nA, P, isd)

In [11]:
class Env2D(object):
    def __init__(self, *args):
        raise NotImplementedError()
    
    def seed(self, seed):
        self.env1.seed(seed)
        self.env2.seed(seed)
    
    def reset(self):
        self.env1.reset()
        self.env2.reset()
    
    @property
    def s(self):
        return self.env1.s + self.env2.s*self.env1.nS
    
    def step(self, A):
        A1, A2 = A % self.nAs[0], A // self.nAs[0]
        S1_, R1, done1, info1 = self.env1.step(A1)
        S2_, R2, done2, info2 = self.env2.step(A2)
        return S1_+S2_*self.env1.nS, R1+R2, (done1 and done2), None

In [12]:
class Env2D_Flattened(discrete.DiscreteEnv):
    def __init__(self, env: Env2D):
        self._env = env # should not be used after flattening
        nS, nA = env.env1.nS * env.env2.nS, env.env1.nA * env.env2.nA
        isd = np.outer(env.env2.isd, env.env1.isd).reshape(-1)
        P = {s : {a : [] for a in range(nA)} for s in range(nS)}
        for s in range(nS):
            s1, s2 = s % env.nSs[0], s // env.nSs[0]
            for a in range(nA):
                a1, a2 = a % env.nAs[0], a // env.nAs[0]
                for (p1, s1_, r1, done1), (p2, s2_, r2, done2) in itertools.product(env.env1.P[s1][a1], env.env2.P[s2][a2]):
                    P[s][a].append((p1*p2, s1_+s2_*env.env1.nS, r1+r2, done1 and done2))
        super().__init__(nS, nA, P, isd)
    
    @property
    def nSs(self):
        return self._env.nSs
    
    @property
    def nAs(self):
        return self._env.nAs
    
    def set_state(self, s):
        self.s = s

    def encode_action(self, a):
        return a % self._env.nAs[0], a // self._env.nAs[0]
    
    def featurize(self, factored=False):
        if factored:
            dA = 1 + len(self.encode_action(0))
            nS, nA = self.nS, self.nA
            dSA = nS*dA
            P = np.zeros((nS, nA, nS))
            R = np.zeros((nS, nA))
            X = np.zeros((nS, nA, dSA))
            for s, actions in self.P.items():
                for a, futures in actions.items():
                    R[s, a] = 0
                    for p, s_, r, done in futures:
                        P[s, a, s_] = p
                        R[s, a] += r*p
                        a1, a2 = self.encode_action(a)
                        X[s, a, s*dA] = 1
                        X[s, a, s*dA+1] = a1
                        X[s, a, s*dA+2] = a2

            X = X.reshape((nS*nA, dSA))
            P = P.reshape((nS*nA, nS))
            R = R.reshape((nS*nA))
            return P, R, X
        else:
            nS, nA = self.nS, self.nA
            dSA = nS*nA
            P = np.zeros((nS, nA, nS))
            R = np.zeros((nS, nA))
            X = np.zeros((nS, nA, dSA))
            for s, actions in self.P.items():
                for a, futures in actions.items():
                    R[s, a] = 0
                    for p, s_, r, done in futures:
                        P[s, a, s_] = p
                        R[s, a] += r*p
                        X[s, a, s*nA+a] = 1

            X = X.reshape((nS*nA, dSA))
            P = P.reshape((nS*nA, nS))
            R = R.reshape((nS*nA))
            return P, R, X

### Bandit (single timestep)

In [13]:
class Bandit(MDP_Env):
    def __init__(self, nA, R):
        nS = 2
        self.nS, self.nA = nS, nA
        S_terminal = self.S_terminal = [1]
        isd = self.isd = np.array([1, 0])
        p_transition = self.p_transition = np.tile(np.array([0,1]), (nS, nA, 1))
        p_reward = self.p_reward = np.zeros((nS, nA))
        p_reward[0, :] = np.array(R)
        super().__init__(p_transition, p_reward, S_terminal, isd)

In [14]:
class Bandit2D(Env2D):
    def __init__(self, nAs, Rs):
        self.env1 = Bandit(nAs[0], Rs[0])
        self.env2 = Bandit(nAs[1], Rs[1])
        self.nSs = (2,2)
        self.nAs, self.Rs = nAs, Rs
        self.nS, self.nA = self.env1.nS * self.env2.nS, self.env1.nA * self.env2.nA

### Chain

In [15]:
# this version of chain allows moving left or right (with probability slipping)
# and only has a reward at the terminal transition
class Chain(MDP_Env):
    def __init__(self, nS=7, nA=4, p_slip=0):
        self.nS, self.nA = nS, nA
        S_terminal = self.S_terminal = [nS-1]
        p_transition = self.p_transition = np.zeros((nS, nA, nS))
        for s in range(nS):
            for a in range(nA):
                if s in S_terminal:
                    p_transition[s,a,s] = 1
                else:
                    if a % 2 == 0: # Left
                        p_transition[s,a,max(s-1, 0)] = 1-p_slip
                        p_transition[s,a,min(s+1, nS-1)] = p_slip
                    else: # Right
                        p_transition[s,a,min(s+1, nS-1)] = 1-p_slip
                        p_transition[s,a,max(s-1, 0)] = p_slip

        p_reward = np.zeros((nS, nA, nS))
        p_reward[nS-2, :, nS-1] = 1

        super().__init__(p_transition, p_reward, S_terminal)

In [16]:
class Chain2D(Env2D):
    def __init__(self, nSs=(3,3), nAs=(2,2), p_slip=(0,0)):
        self.env1 = Chain(nSs[0], nAs[0], p_slip=p_slip[0])
        self.env2 = Chain(nSs[1], nAs[1], p_slip=p_slip[1])
        self.nSs, self.nAs = nSs, nAs
        self.nS, self.nA = self.env1.nS * self.env2.nS, self.env1.nA * self.env2.nA

## Plotting

In [17]:
plot_Q_learning_curves = helpers.plot_Q_learning_curves

In [18]:
import matplotlib.colors as mcolors

def plot_Q_learning_curves(Qs, Q_star=None, zeta=0.0, lw=1.5):
    def flip(items, ncol):
        return itertools.chain(*[items[i::ncol] for i in range(ncol)])
    
    _, nS, nA = Qs.shape
    Q_history = Qs.reshape((len(Qs), -1))
    for s in range(nS):
        for a in range(nA):
            idx = (nA*s+a)
            plt.plot(Q_history[:,idx], 
                     color=(list(mcolors.TABLEAU_COLORS)*100)[s], 
                     alpha=1.0-(a//2)*0.4,
                     lw=lw,
                     ls=[':', '--', '-.', '-'][a], 
                     label='$Q(s_{{{}}}, a={{{}}})$'.format(s,a))
    plt.xlabel('num of transitions/updates')
    plt.ylabel('Q-value')
    handles, labels = plt.gca().get_legend_handles_labels()
    plt.legend(flip(handles, nA), flip(labels, nA), bbox_to_anchor=(1.04,1), loc="upper left") #, ncol=nA
    
    if Q_star is not None:
        V_star = Q_star.max(axis=1)
        Q_cutoff = (1 - zeta) * V_star
        for s, v in enumerate(Q_cutoff):
            plt.axhline(v, alpha=0.6, lw=0.75,
                        c=(list(mcolors.TABLEAU_COLORS)*100)[s])
#     plt.show()
    return


## Experiments

### Simpler Chain (2 states) - Realizability

In [62]:
env = Chain(2,2, p_slip=0)
nS, nA = env.nS, env.nA

π0 = np.array([[1,0], [1,0]])
π1 = np.array([[0,1], [1,0]])

# True Q-values
V0 = algos.policy_eval(env, π0, gamma)
Q0 = algos.V2Q(env, V0, gamma)
V1 = algos.policy_eval(env, π1, gamma)
Q1 = algos.V2Q(env, V1, gamma)
print('π0', π0[0])
print('V0', V0[0])
print('Q0', Q0[0])
print('π0', π1[0])
print('V0', V1[0])
print('Q0', Q1[0])

π0 [1 0]
V0 0.0
Q0 [0. 1.]
π0 [0 1]
V0 1.0
Q0 [0.9 1. ]


#### Additive R, Deterministic P, Deterministic Optimal π Factored wrt factorized states

In [59]:
env = Chain2D((2,2), (2,2), p_slip=(0,0))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize(factored=True)

π = np.array([[0,0,0,1]]*env_flat.nS) # optimal policy

# True Q-values
Q_true = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_true)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_true.reshape(-1))
Q_true_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_true_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_true_approx - Q_true))))
print()

Q_1 = algos.V2Q(env.env1, algos.policy_eval(env.env1, np.array([[0,1],[0,1]]), gamma), gamma)
Q_2 = algos.V2Q(env.env2, algos.policy_eval(env.env2, np.array([[0,1],[0,1]]), gamma), gamma)
print('Q_1')
print(Q_1)
print('Q_2')
print(Q_2)

Q_kron = (
    np.kron(np.ones((4,1)), Q_1.reshape(1,4)) + 
    np.kron(np.ones((1,4)), Q_2.reshape(4,1))
).reshape((2,2,2,2)).transpose((0,2,1,3)).reshape((4,4)) # re-layout block matrix
print('Q_kron')
print(Q_kron)
print('RMSE:', np.sqrt(np.mean(np.square(Q_kron - Q_true))))
print()

True Q-values
[[1.8 1.9 1.9 2. ]
 [0.9 0.9 1.  1. ]
 [0.9 1.  0.9 1. ]
 [0.  0.  0.  0. ]]

Linear approx Q-values
[1.8 0.1 0.1 0.9 0.  0.1 0.9 0.1 0.  0.  0.  0. ]
[[1.8 1.9 1.9 2. ]
 [0.9 0.9 1.  1. ]
 [0.9 1.  0.9 1. ]
 [0.  0.  0.  0. ]]

RMSE: 4.002966042486721e-16

Q_1
[[0.9 1. ]
 [0.  0. ]]
Q_2
[[0.9 1. ]
 [0.  0. ]]
Q_kron
[[1.8 1.9 1.9 2. ]
 [0.9 0.9 1.  1. ]
 [0.9 1.  0.9 1. ]
 [0.  0.  0.  0. ]]
RMSE: 0.0



#### Additive R, Deterministic P, Deterministic Non-optimal π Not Factored wrt factored states

In [69]:
env = Chain2D((2,2), (2,2), p_slip=(0., 0.))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize(factored=True)

π = np.eye(4)[np.random.default_rng(seed=2).random((4,4)).argmax(axis=1)]
π = np.array([
    [0,0,1,0],
    [0,1,0,0],
    [0,0,0,1],
    [0,1,0,0],
])
print('π')
print(π)
print()

# True Q-values
Q_true = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_true)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_true.reshape(-1))
Q_true_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_true_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_true_approx - Q_true))))
print()

π
[[0 0 1 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 1 0 0]]

True Q-values
[[1.71 1.   1.9  2.  ]
 [0.   0.   1.   1.  ]
 [0.9  1.   0.9  1.  ]
 [0.   0.   0.   0.  ]]

Linear approx Q-values
[ 1.5075 -0.305   0.595  -0.      0.      1.      0.9     0.1     0.
  0.      0.     -0.    ]
[[ 1.5075  1.2025  2.1025  1.7975]
 [-0.     -0.      1.      1.    ]
 [ 0.9     1.      0.9     1.    ]
 [ 0.      0.     -0.      0.    ]]

RMSE: 0.10125



In [75]:
π_444x = np.array([
    [0,0,0,1],
    [0,0,0,1],
    [0,0,0,1],
    [1,0,0,0],
])
π_414x = np.array([
    [0,0,0,1],
    [1,0,0,0],
    [0,0,0,1],
    [1,0,0,0],
])
π_212x = np.array([
    [0,1,0,0],
    [1,0,0,0],
    [0,1,0,0],
    [1,0,0,0],
])
π_231x = np.array([
    [0,1,0,0],
    [0,0,1,0],
    [1,0,0,0],
    [1,0,0,0],
])
π_211x = np.array([
    [0,1,0,0],
    [1,0,0,0],
    [1,0,0,0],
    [1,0,0,0],
])
π_132x = np.array([
    [1,0,0,0],
    [0,0,1,0],
    [0,1,0,0],
    [1,0,0,0],
])
π_131x = np.array([
    [1,0,0,0],
    [0,0,1,0],
    [1,0,0,0],
    [1,0,0,0],
])
π_111x = np.array([
    [1,0,0,0],
    [1,0,0,0],
    [1,0,0,0],
    [1,0,0,0],
])

In [76]:
for π in [π_444x, π_414x, π_231x, π_212x, π_211x, π_132x, π_131x, π_111x]:
    print('======')
    env = Chain2D((2,2), (2,2), p_slip=(0., 0.))
    env_flat = Env2D_Flattened(env)
    nS, nA = env_flat.nS, env_flat.nA
    (P,R,X) = env_flat.featurize(factored=True)

    print('π')
    print(π)
    print()

    # True Q-values
    Q_true = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
    print('True Q-values')
    print(Q_true)
    print()

    # Linear approx Q-values
    w = (np.linalg.pinv(X) @ Q_true.reshape(-1))
    Q_true_approx = (X @ w).reshape((nS, nA))
    print('Linear approx Q-values')
    print(w)
    print(Q_true_approx)
    print()
    print('RMSE:', np.sqrt(np.mean(np.square(Q_true_approx - Q_true))))
    print()

π
[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [1 0 0 0]]

True Q-values
[[1.8 1.9 1.9 2. ]
 [0.9 0.9 1.  1. ]
 [0.9 1.  0.9 1. ]
 [0.  0.  0.  0. ]]

Linear approx Q-values
[1.8 0.1 0.1 0.9 0.  0.1 0.9 0.1 0.  0.  0.  0. ]
[[1.8 1.9 1.9 2. ]
 [0.9 0.9 1.  1. ]
 [0.9 1.  0.9 1. ]
 [0.  0.  0.  0. ]]

RMSE: 4.002966042486721e-16

π
[[0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 0]]

True Q-values
[[1.8 1.  1.9 2. ]
 [0.  0.  1.  1. ]
 [0.9 1.  0.9 1. ]
 [0.  0.  0.  0. ]]

Linear approx Q-values
[ 1.575 -0.35   0.55  -0.     0.     1.     0.9    0.1    0.     0.
  0.    -0.   ]
[[ 1.575  1.225  2.125  1.775]
 [-0.    -0.     1.     1.   ]
 [ 0.9    1.     0.9    1.   ]
 [ 0.     0.    -0.     0.   ]]

RMSE: 0.11250000000000002

π
[[0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 0]]

True Q-values
[[1.71 1.9  1.   2.  ]
 [0.9  0.9  1.   1.  ]
 [0.   1.   0.   1.  ]
 [0.   0.   0.   0.  ]]

Linear approx Q-values
[ 1.5075  0.595  -0.305   0.9     0.      0.1     0.      1.     -0.
  0.      0.     -0.    ]
[[ 1.5

#### Change transitions

In [83]:
env = Chain2D((2,2), (2,2), p_slip=(0., 0.))
env_flat = Env2D_Flattened(env)
env_flat.P[0][3] = [(0.9, 3, 2.0, True), (0.1, 2, 1.0, True)]

nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize(factored=True)

π = np.array([[0,0,0,1]]*env_flat.nS) # optimal policy

# True Q-values
Q_true = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_true)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_true.reshape(-1))
Q_true_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_true_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_true_approx - Q_true))))
print()

True Q-values
[[1.71 1.9  1.9  1.9 ]
 [0.9  0.9  1.   1.  ]
 [0.9  1.   0.9  1.  ]
 [0.   0.   0.   0.  ]]

Linear approx Q-values
[ 1.7575  0.095   0.095   0.9     0.      0.1     0.9     0.1    -0.
  0.     -0.      0.    ]
[[ 1.7575  1.8525  1.8525  1.9475]
 [ 0.9     0.9     1.      1.    ]
 [ 0.9     1.      0.9     1.    ]
 [ 0.     -0.      0.      0.    ]]

RMSE: 0.023749999999999938



#### Change rewards

In [87]:
env_flat.P

{0: {0: [(1.0, 0, 0.0, False)],
  1: [(1.0, 1, 1.0, False)],
  2: [(1.0, 2, 1.0, False)],
  3: [(1.0, 3, 2.0, True)]},
 1: {0: [(1.0, 1, 0.0, False)],
  1: [(1.0, 1, 0.0, False)],
  2: [(1.0, 3, 1.0, True)],
  3: [(1.0, 3, 1.0, True)]},
 2: {0: [(1.0, 2, 0.0, False)],
  1: [(1.0, 3, 1.0, True)],
  2: [(1.0, 2, 0.0, False)],
  3: [(1.0, 3, 1.0, True)]},
 3: {0: [(1.0, 3, 0.0, True)],
  1: [(1.0, 3, 0.0, True)],
  2: [(1.0, 3, 0.0, True)],
  3: [(1.0, 3, 0.0, True)]}}

In [92]:
env = Chain2D((2,2), (2,2), p_slip=(0., 0.))
env_flat = Env2D_Flattened(env)
env_flat.P = {
    0: {0: [(1.0, 0, 7.0, False)],
        1: [(1.0, 1, 3.0, False)],
        2: [(1.0, 2, 2.0, False)],
        3: [(1.0, 3, 1.5, True)]},
    1: {0: [(1.0, 1, 0.0, False)],
        1: [(1.0, 1, 0.0, False)],
        2: [(1.0, 3, 4.0, True)],
        3: [(1.0, 3, 4.0, True)]},
    2: {0: [(1.0, 2, 0.0, False)],
        1: [(1.0, 3, 1.0, True)],
        2: [(1.0, 2, 0.0, False)],
        3: [(1.0, 3, 1.0, True)]},
    3: {0: [(1.0, 3, 0.0, True)],
        1: [(1.0, 3, 0.0, True)],
        2: [(1.0, 3, 0.0, True)],
        3: [(1.0, 3, 0.0, True)]}}

nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize(factored=True)

π = np.array([[0,0,0,1]]*env_flat.nS)

# True Q-values
Q_true = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, 1), 1)
print('True Q-values')
print(Q_true)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_true.reshape(-1))
Q_true_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_true_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_true_approx - Q_true))))
print()

# To measure factored rewards violation, set discount to 0

# True Q-values
Q_true = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, 0), 0)
print('True Q-values')
print(Q_true)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_true.reshape(-1))
Q_true_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_true_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_true_approx - Q_true))))
print()

True Q-values
[[8.5 7.  3.  1.5]
 [4.  4.  4.  4. ]
 [1.  1.  1.  1. ]
 [0.  0.  0.  0. ]]

Linear approx Q-values
[ 8.5 -1.5 -5.5  4.   0.   0.   1.  -0.   0.   0.   0.   0. ]
[[8.5 7.  3.  1.5]
 [4.  4.  4.  4. ]
 [1.  1.  1.  1. ]
 [0.  0.  0.  0. ]]

RMSE: 1.280973189656179e-15

True Q-values
[[7.  3.  2.  1.5]
 [0.  0.  4.  4. ]
 [0.  1.  0.  1. ]
 [0.  0.  0.  0. ]]

Linear approx Q-values
[ 6.125 -2.25  -3.25  -0.     0.     4.     0.     1.    -0.     0.
  0.    -0.   ]
[[ 6.125  3.875  2.875  0.625]
 [-0.    -0.     4.     4.   ]
 [ 0.     1.    -0.     1.   ]
 [ 0.     0.    -0.    -0.   ]]

RMSE: 0.4375



### Chains - Realizability

#### Additive R, Deterministic P, Deterministic Optimal π Factored wrt factorized states

In [19]:
env = Chain2D((3,3), (2,2), p_slip=(0,0))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize(factored=True)

π = np.array([[0,0,0,1]]*env_flat.nS) # optimal policy

# True Q-values
Q_true = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_true)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_true.reshape(-1))
Q_true_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_true_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_true_approx - Q_true))))
print()

Q_1 = algos.V2Q(env.env1, algos.policy_eval(env.env1, np.array([[0,1],[0,1],[0,1]]), gamma), gamma)
Q_2 = algos.V2Q(env.env2, algos.policy_eval(env.env2, np.array([[0,1],[0,1],[0,1]]), gamma), gamma)
print('Q_1')
print(Q_1)
print('Q_2')
print(Q_2)

Q_kron = (
    np.kron(np.ones((6,1)), Q_1.reshape(1,6)) + 
    np.kron(np.ones((1,6)), Q_2.reshape(6,1))
).reshape((3,2,3,2)).transpose((0,2,1,3)).reshape((9,4)) # re-layout block matrix
print('Q_kron')
print(Q_kron)
print('RMSE:', np.sqrt(np.mean(np.square(Q_kron - Q_true))))
print()

True Q-values
[[1.62 1.71 1.71 1.8 ]
 [1.62 1.81 1.71 1.9 ]
 [0.81 0.81 0.9  0.9 ]
 [1.62 1.71 1.81 1.9 ]
 [1.62 1.81 1.81 2.  ]
 [0.81 0.81 1.   1.  ]
 [0.81 0.9  0.81 0.9 ]
 [0.81 1.   0.81 1.  ]
 [0.   0.   0.   0.  ]]

Linear approx Q-values
[ 1.62  0.09  0.09  1.62  0.19  0.09  0.81 -0.    0.09  1.62  0.09  0.19
  1.62  0.19  0.19  0.81 -0.    0.19  0.81  0.09 -0.    0.81  0.19 -0.
  0.   -0.    0.  ]
[[ 1.62  1.71  1.71  1.8 ]
 [ 1.62  1.81  1.71  1.9 ]
 [ 0.81  0.81  0.9   0.9 ]
 [ 1.62  1.71  1.81  1.9 ]
 [ 1.62  1.81  1.81  2.  ]
 [ 0.81  0.81  1.    1.  ]
 [ 0.81  0.9   0.81  0.9 ]
 [ 0.81  1.    0.81  1.  ]
 [ 0.   -0.    0.   -0.  ]]

RMSE: 5.350102692277522e-16

Q_1
[[0.81 0.9 ]
 [0.81 1.  ]
 [0.   0.  ]]
Q_2
[[0.81 0.9 ]
 [0.81 1.  ]
 [0.   0.  ]]
Q_kron
[[1.62 1.71 1.71 1.8 ]
 [1.62 1.81 1.71 1.9 ]
 [0.81 0.81 0.9  0.9 ]
 [1.62 1.71 1.81 1.9 ]
 [1.62 1.81 1.81 2.  ]
 [0.81 0.81 1.   1.  ]
 [0.81 0.9  0.81 0.9 ]
 [0.81 1.   0.81 1.  ]
 [0.   0.   0.   0.  ]]
RMSE: 0.0



#### Non-additive R; Deterministic P, Deterministic Optimal π Factored wrt factorized states

- Not going to repeat the same analysis already done on the bandit 2D setting
- Intuition: base case of induction proof will break, i.e. terminating state

In [20]:
R_changed = 4 # not equal to 1 + 1

env = Chain2D((3,3), (2,2), p_slip=(0, 0))
env_flat = Env2D_Flattened(env)
for s in range(env_flat.nS):
    for a in range(env_flat.nA):
        futures = env_flat.P[s][a]
        for idx, future in enumerate(futures):
            (p, s_, r, done) = future
            if r == 2:
                futures[idx] = p, s_, R_changed, done


nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize(factored=True)

π = np.array([[0,0,0,1]]*env_flat.nS)

# True Q-values
Q_true = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_true)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_true.reshape(-1))
Q_true_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_true_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_true_approx - Q_true))))
print()

True Q-values
[[3.24 1.71 1.71 3.6 ]
 [3.24 1.81 1.71 1.9 ]
 [0.81 0.81 0.9  0.9 ]
 [3.24 1.71 1.81 1.9 ]
 [3.24 1.81 1.81 4.  ]
 [0.81 0.81 1.   1.  ]
 [0.81 0.9  0.81 0.9 ]
 [0.81 1.   0.81 1.  ]
 [0.   0.   0.   0.  ]]

Linear approx Q-values
[ 2.385  0.18   0.18   2.835 -0.62  -0.72   0.81  -0.     0.09   2.835
 -0.72  -0.62   2.335  0.38   0.38   0.81  -0.     0.19   0.81   0.09
 -0.     0.81   0.19  -0.     0.     0.    -0.   ]
[[ 2.385  2.565  2.565  2.745]
 [ 2.835  2.215  2.115  1.495]
 [ 0.81   0.81   0.9    0.9  ]
 [ 2.835  2.115  2.215  1.495]
 [ 2.335  2.715  2.715  3.095]
 [ 0.81   0.81   1.     1.   ]
 [ 0.81   0.9    0.81   0.9  ]
 [ 0.81   1.     0.81   1.   ]
 [ 0.     0.    -0.    -0.   ]]

RMSE: 0.4568126287415638



#### Additive R, Deterministic P, Deterministic Non-optimal π Factored wrt factorized states

In [21]:
env = Chain2D((3,3), (2,2), p_slip=(0,0))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize(factored=True)

π_1 = np.array([[1,0],[0,1],[0,1]])
π_2 = np.array([[0,1],[1,0],[0,1]])
π = np.outer(π_1.reshape(-1), π_2.reshape(-1)).T.reshape((3,2,3,2)).transpose((0,2,1,3)).reshape((9,4))

# True Q-values
Q_true = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_true)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_true.reshape(-1))
Q_true_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_true_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_true_approx - Q_true))))
print()

Q_1 = algos.V2Q(env.env1, algos.policy_eval(env.env1, π_1, gamma), gamma)
Q_2 = algos.V2Q(env.env2, algos.policy_eval(env.env2, π_2, gamma), gamma)
print('Q_1')
print(Q_1)
print('Q_2')
print(Q_2)

Q_kron = (
    np.kron(np.ones((6,1)), Q_1.reshape(1,6)) + 
    np.kron(np.ones((1,6)), Q_2.reshape(6,1))
).reshape((3,2,3,2)).transpose((0,2,1,3)).reshape((9,4)) # re-layout block matrix
print('Q_kron')
print(Q_kron)
print('RMSE:', np.sqrt(np.mean(np.square(Q_kron - Q_true))))
print()

True Q-values
[[0.  0.9 0.  0.9]
 [0.  1.  0.  1. ]
 [0.  0.  0.  0. ]
 [0.  0.9 1.  1.9]
 [0.  1.  1.  2. ]
 [0.  0.  1.  1. ]
 [0.  0.9 0.  0.9]
 [0.  1.  0.  1. ]
 [0.  0.  0.  0. ]]

Linear approx Q-values
[-0.   0.9 -0.   0.   1.  -0.   0.  -0.   0.  -0.   0.9  1.  -0.   1.
  1.  -0.  -0.   1.   0.   0.9 -0.   0.   1.  -0.   0.  -0.   0. ]
[[-0.   0.9 -0.   0.9]
 [ 0.   1.  -0.   1. ]
 [ 0.  -0.   0.   0. ]
 [-0.   0.9  1.   1.9]
 [-0.   1.   1.   2. ]
 [-0.  -0.   1.   1. ]
 [ 0.   0.9 -0.   0.9]
 [ 0.   1.  -0.   1. ]
 [ 0.  -0.   0.  -0. ]]

RMSE: 3.5709657172820817e-16

Q_1
[[0.  0.9]
 [0.  1. ]
 [0.  0. ]]
Q_2
[[0. 0.]
 [0. 1.]
 [0. 0.]]
Q_kron
[[0.  0.9 0.  0.9]
 [0.  1.  0.  1. ]
 [0.  0.  0.  0. ]
 [0.  0.9 1.  1.9]
 [0.  1.  1.  2. ]
 [0.  0.  1.  1. ]
 [0.  0.9 0.  0.9]
 [0.  1.  0.  1. ]
 [0.  0.  0.  0. ]]
RMSE: 0.0



#### Additive R, Stochastic P, Deterministic Non-optimal π Factored wrt factorized states

In [22]:
env = Chain2D((3,3), (2,2), p_slip=(0.1, 0.2))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize(factored=True)

π_1 = np.array([[1,0],[0,1],[0,1]])
π_2 = np.array([[0,1],[1,0],[0,1]])
π = np.outer(π_1.reshape(-1), π_2.reshape(-1)).T.reshape((3,2,3,2)).transpose((0,2,1,3)).reshape((9,4))

# True Q-values
Q_true = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_true)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_true.reshape(-1))
Q_true_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_true_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_true_approx - Q_true))))
print()

Q_1 = algos.V2Q(env.env1, algos.policy_eval(env.env1, π_1, gamma), gamma)
Q_2 = algos.V2Q(env.env2, algos.policy_eval(env.env2, π_2, gamma), gamma)
print('Q_1')
print(Q_1)
print('Q_2')
print(Q_2)

Q_kron = (
    np.kron(np.ones((6,1)), Q_1.reshape(1,6)) + 
    np.kron(np.ones((1,6)), Q_2.reshape(6,1))
).reshape((3,2,3,2)).transpose((0,2,1,3)).reshape((9,4)) # re-layout block matrix
print('Q_kron')
print(Q_kron)
print('RMSE:', np.sqrt(np.mean(np.square(Q_kron - Q_true))))
print()

True Q-values
[[0.88694 1.24318 0.92275 1.27899]
 [0.90234 1.38172 0.93815 1.41753]
 [0.44164 0.44164 0.47745 0.47745]
 [0.98907 1.34531 1.33124 1.68748]
 [1.00446 1.48384 1.34663 1.82602]
 [0.54377 0.54377 0.88594 0.88594]
 [0.4453  0.80154 0.4453  0.80154]
 [0.46069 0.94008 0.46069 0.94008]
 [0.      0.      0.      0.     ]]

Linear approx Q-values
[ 0.88694  0.35624  0.03581  0.90234  0.47938  0.03581  0.44164 -0.
  0.03581  0.98907  0.35624  0.34218  1.00446  0.47938  0.34218  0.54377
 -0.       0.34218  0.4453   0.35624 -0.       0.46069  0.47938 -0.
  0.       0.      -0.     ]
[[ 0.88694  1.24318  0.92275  1.27899]
 [ 0.90234  1.38172  0.93815  1.41753]
 [ 0.44164  0.44164  0.47745  0.47745]
 [ 0.98907  1.34531  1.33124  1.68748]
 [ 1.00446  1.48384  1.34663  1.82602]
 [ 0.54377  0.54377  0.88594  0.88594]
 [ 0.4453   0.80154  0.4453   0.80154]
 [ 0.46069  0.94008  0.46069  0.94008]
 [ 0.       0.      -0.       0.     ]]

RMSE: 7.771354952575026e-12

Q_1
[[0.4453  0.80154]
 [0

#### Additive R, Stochastic P, Stochastic Non-optimal π Factored wrt factored states

In [23]:
env = Chain2D((3,3), (2,2), p_slip=(0.1, 0.3))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize(factored=True)

π_1 = np.array([[0.2, 0.8], [0.7, 0.3], [0.6, 0.4]])
π_2 = np.array([[0.9, 0.1], [0.4, 0.6], [0.1, 0.9]])
π = np.outer(π_1.reshape(-1), π_2.reshape(-1)).T.reshape((3,2,3,2)).transpose((0,2,1,3)).reshape((9,4))

# True Q-values
Q_true = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_true)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_true.reshape(-1))
Q_true_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_true_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_true_approx - Q_true))))
print()

Q_1 = algos.V2Q(env.env1, algos.policy_eval(env.env1, π_1, gamma), gamma)
Q_2 = algos.V2Q(env.env2, algos.policy_eval(env.env2, π_2, gamma), gamma)
print('Q_1')
print(Q_1)
print('Q_2')
print(Q_2)

Q_kron = (
    np.kron(np.ones((6,1)), Q_1.reshape(1,6)) + 
    np.kron(np.ones((1,6)), Q_2.reshape(6,1))
).reshape((3,2,3,2)).transpose((0,2,1,3)).reshape((9,4)) # re-layout block matrix
print('Q_kron')
print(Q_kron)
print('RMSE:', np.sqrt(np.mean(np.square(Q_kron - Q_true))))
print()

True Q-values
[[1.1431  1.20919 1.2127  1.27879]
 [1.17982 1.53965 1.24942 1.60925]
 [0.58463 0.58463 0.65423 0.65423]
 [1.23117 1.29726 1.4182  1.48429]
 [1.26789 1.62772 1.45492 1.81475]
 [0.6727  0.6727  0.85973 0.85973]
 [0.55847 0.62456 0.55847 0.62456]
 [0.59519 0.95502 0.59519 0.95502]
 [0.      0.      0.      0.     ]]

Linear approx Q-values
[ 1.1431   0.06609  0.0696   1.17982  0.35983  0.0696   0.58463 -0.
  0.0696   1.23117  0.06609  0.18703  1.26789  0.35983  0.18703  0.6727
 -0.       0.18703  0.55847  0.06609 -0.       0.59519  0.35983 -0.
  0.       0.      -0.     ]
[[ 1.1431   1.20919  1.2127   1.27879]
 [ 1.17982  1.53965  1.24942  1.60925]
 [ 0.58463  0.58463  0.65423  0.65423]
 [ 1.23117  1.29726  1.4182   1.48429]
 [ 1.26789  1.62772  1.45492  1.81475]
 [ 0.6727   0.6727   0.85973  0.85973]
 [ 0.55847  0.62456  0.55847  0.62456]
 [ 0.59519  0.95502  0.59519  0.95502]
 [ 0.       0.      -0.       0.     ]]

RMSE: 1.86370766540755e-12

Q_1
[[0.55847 0.62456]
 [0.5

#### Additive R, Deterministic P, Deterministic Non-optimal π Not Factored wrt factored states

In [26]:
env = Chain2D((3,3), (2,2), p_slip=(0., 0.))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize(factored=True)

π = np.eye(4)[np.random.default_rng(seed=0).random((9,4)).argmax(axis=1)]
# π = np.array([
#     [0,0,0,1],
#     [1,0,0,0],
#     [0,0,0,1],
#     [0,1,0,0],
#     [1,0,0,0],
#     [0,1,0,0],
#     [0,0,1,0],
#     [1,0,0,0],
#     [0,0,1,0.],
# ])
# π = np.array([
#     [1,0,0,0],
#     [0,1,0,0],
#     [0,0,0,1],
#     [0,1,0,0],
#     [1,0,0,0],
#     [0,1,0,0],
#     [0,0,1,0],
#     [1,0,0,0],
#     [0,0,1,0.],
# ])
print('π')
print(π)
print()

# True Q-values
Q_true = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_true)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_true.reshape(-1))
Q_true_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_true_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_true_approx - Q_true))))
print()

π
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]

True Q-values
[[0.  0.9 0.  0. ]
 [0.  1.  0.  1.9]
 [0.  0.  0.9 0.9]
 [0.  0.9 1.  1. ]
 [0.  1.  1.  2. ]
 [0.  0.  1.  1. ]
 [0.  0.  0.  0. ]
 [0.  1.  0.  1. ]
 [0.  0.  0.  0. ]]

Linear approx Q-values
[ 0.225  0.45  -0.45  -0.225  1.45   0.45  -0.    -0.     0.9    0.225
  0.45   0.55  -0.     1.     1.    -0.    -0.     1.     0.    -0.
  0.     0.     1.    -0.     0.    -0.     0.   ]
[[ 0.225  0.675 -0.225  0.225]
 [-0.225  1.225  0.225  1.675]
 [-0.    -0.     0.9    0.9  ]
 [ 0.225  0.675  0.775  1.225]
 [-0.     1.     1.     2.   ]
 [-0.    -0.     1.     1.   ]
 [ 0.    -0.     0.     0.   ]
 [ 0.     1.    -0.     1.   ]
 [ 0.    -0.     0.     0.   ]]

RMSE: 0.12990381056766578



In [28]:
P, R, X = env_flat.featurize(factored=True)

In [32]:
P0, R0, X0 = env_flat.featurize(factored=False)

In [30]:
R.reshape((9,4))

array([[0., 0., 0., 0.],
       [0., 1., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 1., 1.],
       [0., 1., 1., 2.],
       [0., 0., 1., 1.],
       [0., 0., 0., 0.],
       [0., 1., 0., 1.],
       [0., 0., 0., 0.]])

In [34]:
π

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]])

In [35]:
Pi_w = np.zeros((nS, nS, nA))
for s in range(nS):
    Pi_w[s,s,:] = π[s]
Pi_w = Pi_w.reshape((nS, -1))

In [47]:
Q_h = [R]
for k in range(1):
    Q_h.append(R + gamma * P @ Pi_w @ Q_h[-1])

In [49]:
print(np.array(Q_h).reshape((-1,9,4)))

[[[0.  0.  0.  0. ]
  [0.  1.  0.  1. ]
  [0.  0.  0.  0. ]
  [0.  0.  1.  1. ]
  [0.  1.  1.  2. ]
  [0.  0.  1.  1. ]
  [0.  0.  0.  0. ]
  [0.  1.  0.  1. ]
  [0.  0.  0.  0. ]]

 [[0.  0.9 0.  0. ]
  [0.  1.  0.  1.9]
  [0.  0.  0.9 0.9]
  [0.  0.9 1.  1. ]
  [0.  1.  1.  2. ]
  [0.  0.  1.  1. ]
  [0.  0.  0.  0. ]
  [0.  1.  0.  1. ]
  [0.  0.  0.  0. ]]]


In [58]:
(np.linalg.pinv(X) @ Q_h[0]).reshape((9,-1))

array([[ 0.,  0.,  0.],
       [ 0.,  1., -0.],
       [ 0.,  0., -0.],
       [-0., -0.,  1.],
       [-0.,  1.,  1.],
       [-0., -0.,  1.],
       [ 0., -0.,  0.],
       [ 0.,  1., -0.],
       [ 0., -0.,  0.]])

In [53]:
(X @ (np.linalg.pinv(X) @ Q_h[0])).reshape((9,4))

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  1., -0.,  1.],
       [ 0.,  0., -0.,  0.],
       [-0., -0.,  1.,  1.],
       [-0.,  1.,  1.,  2.],
       [-0., -0.,  1.,  1.],
       [ 0., -0.,  0., -0.],
       [ 0.,  1., -0.,  1.],
       [ 0., -0.,  0.,  0.]])

In [54]:
(X @ (np.linalg.pinv(X) @ Q_h[1])).reshape((9,4))

array([[ 0.225,  0.675, -0.225,  0.225],
       [-0.225,  1.225,  0.225,  1.675],
       [-0.   , -0.   ,  0.9  ,  0.9  ],
       [ 0.225,  0.675,  0.775,  1.225],
       [-0.   ,  1.   ,  1.   ,  2.   ],
       [-0.   , -0.   ,  1.   ,  1.   ],
       [ 0.   , -0.   ,  0.   ,  0.   ],
       [ 0.   ,  1.   , -0.   ,  1.   ],
       [ 0.   , -0.   ,  0.   ,  0.   ]])

### Stochastic P, Stochastic π; Both are perfectly factored with factored state space

In [52]:
p_list = [
    [0.1, 0.2, 0.9],
    [0.7, 0.4, 0.3],
]

env = Chain2D((3,3), (2,2), p_slip=(0.2, 0.2))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize(factored=True)

π = np.array([
    [
        p_list[0][s//3]*p_list[1][s%3], 
        (1-p_list[0][s//3])*p_list[1][s%3],
        p_list[0][s//3]*(1-p_list[1][s%3]), 
        (1-p_list[0][s//3])*(1-p_list[1][s%3]),
    ] for s in range(env_flat.nS)])
print(π)
print('Check π:', π.sum(axis=1))

# True Q-values
Q_star = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_star)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_star.reshape(-1))
Q_star_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_star_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_star_approx - Q_star))))
print()

[[0.07 0.63 0.03 0.27]
 [0.04 0.36 0.06 0.54]
 [0.03 0.27 0.07 0.63]
 [0.14 0.56 0.06 0.24]
 [0.08 0.32 0.12 0.48]
 [0.06 0.24 0.14 0.56]
 [0.63 0.07 0.27 0.03]
 [0.36 0.04 0.54 0.06]
 [0.27 0.03 0.63 0.07]]
Check π: [1. 1. 1. 1. 1. 1. 1. 1. 1.]
True Q-values
[[1.31085 1.42408 1.30743 1.44859]
 [1.35656 1.60693 1.36072 1.66172]
 [0.69039 0.69039 0.76205 0.76205]
 [1.31631 1.41522 1.32929 1.41312]
 [1.37332 1.64323 1.42774 1.80691]
 [0.7332  0.7332  0.9333  0.9333 ]
 [0.33362 0.41243 0.33362 0.41243]
 [0.44588 0.86147 0.44588 0.86147]
 [0.      0.      0.      0.     ]]

Linear approx Q-values
[ 1.30387  0.12719  0.01054  1.3439   0.27569  0.02947  0.69039 -0.
  0.07167  1.32008  0.09137  0.00544  1.346    0.32454  0.10905  0.7332
 -0.       0.2001   0.33362  0.07881  0.       0.44588  0.41559 -0.
  0.       0.      -0.     ]
[[ 1.30387  1.43106  1.31441  1.44161]
 [ 1.3439   1.61959  1.37337  1.64906]
 [ 0.69039  0.69039  0.76205  0.76205]
 [ 1.32008  1.41145  1.32552  1.41689]
 [ 1.34

In [32]:
print('Policy is state-independent')
p_0 = 0.2

env = Chain2D((3,3), (2,2), p_slip=(0.2, 0.2))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize(factored=True)

π = np.array([[p_0*p_0, p_0*(1-p_0), p_0*(1-p_0), (1-p_0)*(1-p_0)]]*env_flat.nS)
print(π)
print('Check π:', π.sum(axis=1))

# True Q-values
Q_star = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_star)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_star.reshape(-1))
Q_star_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_star_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_star_approx - Q_star))))
print()

Policy is state-independent
[[0.04 0.16 0.16 0.64]
 [0.04 0.16 0.16 0.64]
 [0.04 0.16 0.16 0.64]
 [0.04 0.16 0.16 0.64]
 [0.04 0.16 0.16 0.64]
 [0.04 0.16 0.16 0.64]
 [0.04 0.16 0.16 0.64]
 [0.04 0.16 0.16 0.64]
 [0.04 0.16 0.16 0.64]]
Check π: [1. 1. 1. 1. 1. 1. 1. 1. 1.]
True Q-values
[[1.44391 1.51245 1.51245 1.58099]
 [1.48124 1.66178 1.54978 1.73032]
 [0.72196 0.72196 0.7905  0.7905 ]
 [1.48124 1.54978 1.66178 1.73032]
 [1.51858 1.69911 1.69911 1.87964]
 [0.75929 0.75929 0.93982 0.93982]
 [0.72196 0.7905  0.72196 0.7905 ]
 [0.75929 0.93982 0.75929 0.93982]
 [0.      0.      0.      0.     ]]

Linear approx Q-values
[ 1.44391  0.06854  0.06854  1.48124  0.18053  0.06854  0.72196 -0.
  0.06854  1.48124  0.06854  0.18053  1.51858  0.18053  0.18053  0.75929
 -0.       0.18053  0.72196  0.06854 -0.       0.75929  0.18053 -0.
  0.      -0.       0.     ]
[[ 1.44391  1.51245  1.51245  1.58099]
 [ 1.48124  1.66178  1.54978  1.73032]
 [ 0.72196  0.72196  0.7905   0.7905 ]
 [ 1.48124  1.549

In [34]:
np.random.default_rng(seed=0).random(env_flat.nS)

array([0.63696, 0.26979, 0.04097, 0.01653, 0.81327, 0.91276, 0.60664,
       0.7295 , 0.54362])

In [39]:
print('Policy is state-dependent')

env = Chain2D((3,3), (2,2), p_slip=(0.2, 0.2))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize(factored=True)

π = np.array([[p_s*p_s, p_s*(1-p_s), p_s*(1-p_s), (1-p_s)*(1-p_s)] for p_s in (1+np.arange(env_flat.nS))/9])
print(π)
print('Check π:', π.sum(axis=1))

# True Q-values
Q_star = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_star)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_star.reshape(-1))
Q_star_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_star_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_star_approx - Q_star))))
print()

Policy is state-dependent
[[0.01235 0.09877 0.09877 0.79012]
 [0.04938 0.17284 0.17284 0.60494]
 [0.11111 0.22222 0.22222 0.44444]
 [0.19753 0.24691 0.24691 0.30864]
 [0.30864 0.24691 0.24691 0.19753]
 [0.44444 0.22222 0.22222 0.11111]
 [0.60494 0.17284 0.17284 0.04938]
 [0.79012 0.09877 0.09877 0.01235]
 [1.      0.      0.      0.     ]]
Check π: [1. 1. 1. 1. 1. 1. 1. 1. 1.]
True Q-values
[[1.21307 1.30656 1.23167 1.3156 ]
 [1.26125 1.4993  1.29108 1.55325]
 [0.57865 0.57865 0.64064 0.64064]
 [1.24153 1.33313 1.34553 1.4219 ]
 [1.29808 1.55931 1.43838 1.79329]
 [0.64638 0.64638 0.9116  0.9116 ]
 [0.3802  0.45149 0.3802  0.45149]
 [0.48515 0.87129 0.48515 0.87129]
 [0.      0.      0.      0.     ]]

Linear approx Q-values
[ 1.21545  0.08872  0.01382  1.25522  0.25011  0.04189  0.57865 -0.
  0.062    1.24534  0.08398  0.09638  1.27466  0.30807  0.18715  0.64638
 -0.       0.26521  0.3802   0.07129  0.       0.48515  0.38614 -0.
  0.       0.      -0.     ]
[[ 1.21545  1.30417  1.22928

In [45]:
print('Policy is state-dependent')

env = Chain2D((3,3), (2,2), p_slip=(0., 0.))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize(factored=True)

π = np.array([
    [0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    
    [0.25, 0.25, 0.25, 0.25],##
    
    [0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    
    [0.25, 0.25, 0.25, 0.25],##
    [0.1 , 0.2 , 0.3 , 0.4 ],####
    [0.25, 0.25, 0.25, 0.25],####
    [1,0,0,0], ######
])
print(π)
print('Check π:', π.sum(axis=1))

# True Q-values
Q_star = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_star)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_star.reshape(-1))
Q_star_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_star_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_star_approx - Q_star))))
print()

Policy is state-dependent
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.1  0.2  0.3  0.4 ]
 [0.25 0.25 0.25 0.25]
 [1.   0.   0.   0.  ]]
Check π: [1. 1. 1. 1. 1. 1. 1. 1. 1.]
True Q-values
[[1.17265 1.29932 1.30714 1.43268]
 [1.17265 1.58273 1.30714 1.71223]
 [0.58273 0.58273 0.71223 0.71223]
 [1.17265 1.29932 1.61209 1.72544]
 [1.17265 1.58273 1.61209 2.     ]
 [0.58273 0.58273 1.      1.     ]
 [0.61209 0.72544 0.61209 0.72544]
 [0.61209 1.      0.61209 1.     ]
 [0.      0.      0.      0.     ]]

Linear approx Q-values
[ 1.17293  0.12611  0.13392  1.1739   0.40759  0.13199  0.58273 -0.
  0.1295   1.17598  0.12001  0.43278  1.1782   0.39899  0.42835  0.58273
 -0.       0.41727  0.61209  0.11335 -0.       0.61209  0.38791 -0.
  0.      -0.       0.     ]
[[ 1.17293  1.29904  1.30686  1.43296]
 [ 1.1739   1.58149  1.30589  1.71348]
 [ 0.58273  0.58273  0.71223  0.71223]
 [ 1.17598  1.29599

### Uniformly random policy, still factored

In [111]:
env = Chain2D((3,3), (2,2), p_slip=(0, 0))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize()

π = np.array([
    [0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25],
])

# True Q-values
Q_star = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_star)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_star.reshape(-1))
Q_star_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_star_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_star_approx - Q_star))))
print()

True Q-values
[[1.16547 1.29496 1.29496 1.42446]
 [1.16547 1.58273 1.29496 1.71223]
 [0.58273 0.58273 0.71223 0.71223]
 [1.16547 1.29496 1.58273 1.71223]
 [1.16547 1.58273 1.58273 2.     ]
 [0.58273 0.58273 1.      1.     ]
 [0.58273 0.71223 0.58273 0.71223]
 [0.58273 1.      0.58273 1.     ]
 [0.      0.      0.      0.     ]]

Linear approx Q-values
[ 1.16547e+00  1.29496e-01  1.29496e-01  1.16547e+00  4.17266e-01
  1.29496e-01  5.82734e-01 -5.55112e-17  1.29496e-01  1.16547e+00
  1.29496e-01  4.17266e-01  1.16547e+00  4.17266e-01  4.17266e-01
  5.82734e-01 -1.11022e-16  4.17266e-01  5.82734e-01  1.29496e-01
 -1.11022e-16  5.82734e-01  4.17266e-01 -2.22045e-16  0.00000e+00
 -1.64234e-43  1.64239e-43]
[[ 1.16547e+00  1.29496e+00  1.29496e+00  1.42446e+00]
 [ 1.16547e+00  1.58273e+00  1.29496e+00  1.71223e+00]
 [ 5.82734e-01  5.82734e-01  7.12230e-01  7.12230e-01]
 [ 1.16547e+00  1.29496e+00  1.58273e+00  1.71223e+00]
 [ 1.16547e+00  1.58273e+00  1.58273e+00  2.00000e+00]
 [ 5.82734e-0

### Deterministic P, Non-factorizable π

### Non-factorizable policy

In [145]:
env = Chain2D((3,3), (2,2), p_slip=(0,0))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize()

π = np.array([
    [0.1, 0.2, 0.3, 0.4],
    [0.1, 0.2, 0.3, 0.4],
    [0.1, 0.2, 0.3, 0.4],
    [0.1, 0.2, 0.3, 0.4],
    [0.1, 0.2, 0.3, 0.4],
    [0.1, 0.2, 0.3, 0.4],
    [0.1, 0.2, 0.3, 0.4],
    [0.1, 0.2, 0.3, 0.4],
    [0,0,0,1],
])
print('Check π:', π.sum(axis=1))

# True Q-values
Q_star = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_star)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_star.reshape(-1))
Q_star_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_star_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_star_approx - Q_star))))
print()

Check π: [1. 1. 1. 1. 1. 1. 1. 1. 1.]
True Q-values
[[1.36328 1.48446 1.4758  1.59698]
 [1.36328 1.70888 1.4758  1.8214 ]
 [0.70888 0.70888 0.8214  0.8214 ]
 [1.36328 1.48446 1.6544  1.77558]
 [1.36328 1.70888 1.6544  2.     ]
 [0.70888 0.70888 1.      1.     ]
 [0.6544  0.77558 0.6544  0.77558]
 [0.6544  1.      0.6544  1.     ]
 [0.      0.      0.      0.     ]]

Linear approx Q-values
[ 1.36328  0.12118  0.11252  1.36328  0.3456   0.11252  0.70888 -0.
  0.11252  1.36328  0.12118  0.29112  1.36328  0.3456   0.29112  0.70888
 -0.       0.29112  0.6544   0.12118  0.       0.6544   0.3456  -0.
  0.      -0.       0.     ]
[[ 1.36328  1.48446  1.4758   1.59698]
 [ 1.36328  1.70888  1.4758   1.8214 ]
 [ 0.70888  0.70888  0.8214   0.8214 ]
 [ 1.36328  1.48446  1.6544   1.77558]
 [ 1.36328  1.70888  1.6544   2.     ]
 [ 0.70888  0.70888  1.       1.     ]
 [ 0.6544   0.77558  0.6544   0.77558]
 [ 0.6544   1.       0.6544   1.     ]
 [ 0.      -0.       0.       0.     ]]

RMSE: 2.730267774

In [140]:
env = Chain2D((3,3), (2,2), p_slip=(0,0))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize()

π = np.array([
    [0.2, 0.1, 0.3, 0.4],
    [0.2, 0.1, 0.3, 0.4],
    [0.2, 0.1, 0.3, 0.4],
    [0.2, 0.1, 0.3, 0.4],
    [0.2, 0.1, 0.3, 0.4],
    [0.2, 0.1, 0.3, 0.4],
    [0.2, 0.1, 0.3, 0.4],
    [0.2, 0.1, 0.3, 0.4],
    [0.2, 0.1, 0.3, 0.4],
])
print('Check π:', π.sum(axis=1))

# True Q-values
Q_star = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_star)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_star.reshape(-1))
Q_star_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_star_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_star_approx - Q_star))))
print()

Check π: [1. 1. 1. 1. 1. 1. 1. 1. 1.]
True Q-values
[[1.29161 1.42111 1.40413 1.53363]
 [1.29161 1.70888 1.40413 1.8214 ]
 [0.70888 0.70888 0.8214  0.8214 ]
 [1.29161 1.42111 1.58273 1.71223]
 [1.29161 1.70888 1.58273 2.     ]
 [0.70888 0.70888 1.      1.     ]
 [0.58273 0.71223 0.58273 0.71223]
 [0.58273 1.      0.58273 1.     ]
 [0.      0.      0.      0.     ]]

Linear approx Q-values
[ 1.29161  0.1295   0.11252  1.29161  0.41727  0.11252  0.70888 -0.
  0.11252  1.29161  0.1295   0.29112  1.29161  0.41727  0.29112  0.70888
 -0.       0.29112  0.58273  0.1295   0.       0.58273  0.41727 -0.
  0.      -0.       0.     ]
[[ 1.29161  1.42111  1.40413  1.53363]
 [ 1.29161  1.70888  1.40413  1.8214 ]
 [ 0.70888  0.70888  0.8214   0.8214 ]
 [ 1.29161  1.42111  1.58273  1.71223]
 [ 1.29161  1.70888  1.58273  2.     ]
 [ 0.70888  0.70888  1.       1.     ]
 [ 0.58273  0.71223  0.58273  0.71223]
 [ 0.58273  1.       0.58273  1.     ]
 [ 0.      -0.       0.       0.     ]]

RMSE: 3.377091656

In [132]:
env = Chain2D((3,3), (2,2), p_slip=(0,0))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize()

π = np.array([
    [0.04, 0.16, 0.16, 0.64],
    [0.04, 0.16, 0.16, 0.64],
    [0.04, 0.16, 0.16, 0.64],
    [0.04, 0.16, 0.16, 0.64],
    [0.04, 0.16, 0.16, 0.64],
    [0.04, 0.16, 0.16, 0.64],
    [0.04, 0.16, 0.16, 0.64],
    [0.04, 0.16, 0.16, 0.64],
    [0.04, 0.16, 0.16, 0.64],
])
print('Check π:', π.sum(axis=1))

# True Q-values
Q_star = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_star)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_star.reshape(-1))
Q_star_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_star_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_star_approx - Q_star))))
print()

Check π: [1. 1. 1. 1. 1. 1. 1. 1. 1.]
True Q-values
[[1.50174 1.60603 1.60603 1.71031]
 [1.50174 1.75087 1.60603 1.85516]
 [0.75087 0.75087 0.85516 0.85516]
 [1.50174 1.60603 1.75087 1.85516]
 [1.50174 1.75087 1.75087 2.     ]
 [0.75087 0.75087 1.      1.     ]
 [0.75087 0.85516 0.75087 0.85516]
 [0.75087 1.      0.75087 1.     ]
 [0.      0.      0.      0.     ]]

Linear approx Q-values
[ 1.50174  0.10429  0.10429  1.50174  0.24913  0.10429  0.75087 -0.
  0.10429  1.50174  0.10429  0.24913  1.50174  0.24913  0.24913  0.75087
 -0.       0.24913  0.75087  0.10429 -0.       0.75087  0.24913 -0.
  0.      -0.       0.     ]
[[ 1.50174  1.60603  1.60603  1.71031]
 [ 1.50174  1.75087  1.60603  1.85516]
 [ 0.75087  0.75087  0.85516  0.85516]
 [ 1.50174  1.60603  1.75087  1.85516]
 [ 1.50174  1.75087  1.75087  2.     ]
 [ 0.75087  0.75087  1.       1.     ]
 [ 0.75087  0.85516  0.75087  0.85516]
 [ 0.75087  1.       0.75087  1.     ]
 [ 0.      -0.       0.       0.     ]]

RMSE: 6.634990697

In [137]:
env = Chain2D((3,3), (2,2), p_slip=(0,0))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize()

π = np.array([
    [0.01, 0.09, 0.09, 0.81],
    [0.02, 0.08, 0.18, 0.72],
    [0.  , 0.10, 0.  , 0.9 ],
    [0.02, 0.18, 0.08, 0.72],
    [0.04, 0.16, 0.16, 0.64],
    [0.  , 0.20, 0.  , 0.8 ],
    [0, 0, 0.1, 0.9],
    [0, 0, 0.2, 0.8],
    [0,0,0,1],
])
print('Check π:', π.sum(axis=1))

# True Q-values
Q_star = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_star)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_star.reshape(-1))
Q_star_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_star_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_star_approx - Q_star))))
print()

Check π: [1. 1. 1. 1. 1. 1. 1. 1. 1.]
True Q-values
[[1.5263  1.62052 1.62052 1.71473]
 [1.5263  1.76315 1.62052 1.85737]
 [0.76315 0.76315 0.85737 0.85737]
 [1.5263  1.62052 1.76315 1.85737]
 [1.5263  1.76315 1.76315 2.     ]
 [0.76315 0.76315 1.      1.     ]
 [0.76315 0.85737 0.76315 0.85737]
 [0.76315 1.      0.76315 1.     ]
 [0.      0.      0.      0.     ]]

Linear approx Q-values
[ 1.5263   0.09422  0.09422  1.5263   0.23685  0.09422  0.76315 -0.
  0.09422  1.5263   0.09422  0.23685  1.5263   0.23685  0.23685  0.76315
 -0.       0.23685  0.76315  0.09422 -0.       0.76315  0.23685 -0.
  0.      -0.       0.     ]
[[ 1.5263   1.62052  1.62052  1.71473]
 [ 1.5263   1.76315  1.62052  1.85737]
 [ 0.76315  0.76315  0.85737  0.85737]
 [ 1.5263   1.62052  1.76315  1.85737]
 [ 1.5263   1.76315  1.76315  2.     ]
 [ 0.76315  0.76315  1.       1.     ]
 [ 0.76315  0.85737  0.76315  0.85737]
 [ 0.76315  1.       0.76315  1.     ]
 [ 0.      -0.       0.       0.     ]]

RMSE: 1.164873466

In [139]:
env = Chain2D((3,3), (2,2), p_slip=(0,0))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize()

π = np.array([
    [0.01, 0.09, 0.09, 0.81],
    [0.02, 0.08, 0.18, 0.72],
    [0.  , 0.10, 0.  , 0.9 ],
    [0.02, 0.18, 0.08, 0.72],
    [0.04, 0.16, 0.16, 0.64],
    [0.  , 0.20, 0.  , 0.8 ],
    [0, 0, 0.1, 0.9],
#     [0, 0, 0.2, 0.8],
    [0.1, 0.2, 0.3, 0.4],
    [0,0,0,1],
])
print('Check π:', π.sum(axis=1))

# True Q-values
Q_star = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_star)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_star.reshape(-1))
Q_star_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_star_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_star_approx - Q_star))))
print()

Check π: [1. 1. 1. 1. 1. 1. 1. 1. 1.]
True Q-values
[[1.51571 1.61286 1.57441 1.70631]
 [1.51571 1.76315 1.57441 1.85737]
 [0.76315 0.76315 0.85737 0.85737]
 [1.51571 1.61286 1.70731 1.79463]
 [1.51571 1.76315 1.70731 2.     ]
 [0.76315 0.76315 1.      1.     ]
 [0.70731 0.79463 0.70731 0.79463]
 [0.70731 1.      0.70731 1.     ]
 [0.      0.      0.      0.     ]]

Linear approx Q-values
[ 1.50702  0.11452  0.07608  1.50683  0.2652   0.07646  0.76315 -0.
  0.09422  1.51817  0.09223  0.18669  1.5044   0.27007  0.21422  0.76315
 -0.       0.23685  0.70731  0.08732  0.       0.70731  0.29269 -0.
  0.       0.      -0.     ]
[[ 1.50702  1.62155  1.5831   1.69762]
 [ 1.50683  1.77203  1.58329  1.84849]
 [ 0.76315  0.76315  0.85737  0.85737]
 [ 1.51817  1.6104   1.70485  1.79709]
 [ 1.5044   1.77446  1.71862  1.98869]
 [ 0.76315  0.76315  1.       1.     ]
 [ 0.70731  0.79463  0.70731  0.79463]
 [ 0.70731  1.       0.70731  1.     ]
 [ 0.       0.      -0.       0.     ]]

RMSE: 0.005660056

In [136]:
env = Chain2D((3,3), (2,2), p_slip=(0,0))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize()

π = np.array([
    [0.01, 0.09, 0.09, 0.81],
    [0.01, 0.09, 0.09, 0.81],
    [0.01, 0.09, 0.09, 0.81],
    [0.01, 0.09, 0.09, 0.81],
    [0.01, 0.09, 0.09, 0.81],
    [0.01, 0.09, 0.09, 0.81],
    [0.01, 0.09, 0.09, 0.81],
    [0.01, 0.09, 0.09, 0.81],
    [0.01, 0.09, 0.09, 0.81],
])
print('Check π:', π.sum(axis=1))

# True Q-values
Q_star = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_star)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_star.reshape(-1))
Q_star_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_star_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_star_approx - Q_star))))
print()

Check π: [1. 1. 1. 1. 1. 1. 1. 1. 1.]
True Q-values
[[1.56755 1.66432 1.66432 1.76108]
 [1.56755 1.78378 1.66432 1.88054]
 [0.78378 0.78378 0.88054 0.88054]
 [1.56755 1.66432 1.78378 1.88054]
 [1.56755 1.78378 1.78378 2.     ]
 [0.78378 0.78378 1.      1.     ]
 [0.78378 0.88054 0.78378 0.88054]
 [0.78378 1.      0.78378 1.     ]
 [0.      0.      0.      0.     ]]

Linear approx Q-values
[ 1.56755  0.09676  0.09676  1.56755  0.21622  0.09676  0.78378 -0.
  0.09676  1.56755  0.09676  0.21622  1.56755  0.21622  0.21622  0.78378
 -0.       0.21622  0.78378  0.09676 -0.       0.78378  0.21622 -0.
  0.      -0.       0.     ]
[[ 1.56755  1.66432  1.66432  1.76108]
 [ 1.56755  1.78378  1.66432  1.88054]
 [ 0.78378  0.78378  0.88054  0.88054]
 [ 1.56755  1.66432  1.78378  1.88054]
 [ 1.56755  1.78378  1.78378  2.     ]
 [ 0.78378  0.78378  1.       1.     ]
 [ 0.78378  0.88054  0.78378  0.88054]
 [ 0.78378  1.       0.78378  1.     ]
 [ 0.      -0.       0.      -0.     ]]

RMSE: 4.324633668

In [135]:
env = Chain2D((3,3), (2,2), p_slip=(0,0))
env_flat = Env2D_Flattened(env)
nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize()

π = np.array([[5,9,7,1]]*9)/22
print('Check π:', π.sum(axis=1))

# True Q-values
Q_star = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_star)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_star.reshape(-1))
Q_star_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_star_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_star_approx - Q_star))))
print()

Check π: [1. 1. 1. 1. 1. 1. 1. 1. 1.]
True Q-values
[[0.98948 1.12219 1.12594 1.25865]
 [0.98948 1.44659 1.12594 1.58305]
 [0.44659 0.44659 0.58305 0.58305]
 [0.98948 1.12219 1.5429  1.6756 ]
 [0.98948 1.44659 1.5429  2.     ]
 [0.44659 0.44659 1.      1.     ]
 [0.5429  0.6756  0.5429  0.6756 ]
 [0.5429  1.      0.5429  1.     ]
 [0.      0.      0.      0.     ]]

Linear approx Q-values
[ 0.98948  0.13271  0.13646  0.98948  0.4571   0.13646  0.44659 -0.
  0.13646  0.98948  0.13271  0.55341  0.98948  0.4571   0.55341  0.44659
 -0.       0.55341  0.5429   0.13271 -0.       0.5429   0.4571  -0.
  0.       0.      -0.     ]
[[ 0.98948  1.12219  1.12594  1.25865]
 [ 0.98948  1.44659  1.12594  1.58305]
 [ 0.44659  0.44659  0.58305  0.58305]
 [ 0.98948  1.12219  1.5429   1.6756 ]
 [ 0.98948  1.44659  1.5429   2.     ]
 [ 0.44659  0.44659  1.       1.     ]
 [ 0.5429   0.6756   0.5429   0.6756 ]
 [ 0.5429   1.       0.5429   1.     ]
 [ 0.       0.      -0.       0.     ]]

RMSE: 2.356871849

### Non-factorizable transitions

In [70]:
np.set_printoptions(precision=None, suppress=False)

In [25]:
p_slip = 0.2
eps = 0.03

env_flat = Env2D_Flattened(Chain2D((3,3), (2,2), p_slip=(p_slip, p_slip)))
for s in range(env_flat.nS):
    for a in range(env_flat.nA):
        futures = env_flat.P[s][a]
        for idx, future in enumerate(futures):
            (p, _, _, _) = future
            if p == 1:
                pass
            elif p == p_slip*(1-p_slip):
                pass
            elif p == p_slip*p_slip:
                futures[idx] = p_slip*p_slip - eps, *future[1:]
            elif p == (1-p_slip)*(1-p_slip):
                futures[idx] = (1-p_slip)*(1-p_slip) + eps, *future[1:]
            elif p == p_slip:
                futures[idx] = p_slip - eps, *future[1:]
            elif p == 1-p_slip:
                futures[idx] = 1-p_slip + eps, *future[1:]
            else:
                assert False, '{} {} {} {}'.format(s,a,idx,future)

env_flat.P[0][0] = [(0.67, 0, 0.0, False),
   (0.12, 3, 0.0, False),
   (0.20, 1, 0.0, False),
   (0.01, 4, 0.0, False)]

env_flat.P[1] = {0: [#(0.82, 0, 0.0, False),
    (0.67, 0, 0.0, False),
   (0.15, 3, 0.0, False),
   (0.17, 2, 1.0, False),
   (0.01, 5, 1.0, False)],
  1: [(0.17, 0, 0.0, False),
   (0.01, 3, 0.0, False),
   (0.67, 2, 1.0, False),
   (0.15, 5, 1.0, False)],
  2: [(0.15, 0, 0.0, False),
   (0.67, 3, 0.0, False),
   (0.01, 2, 1.0, False),
   (0.17, 5, 1.0, False)],
  3: [(0.01, 0, 0.0, False),
   (0.17, 3, 0.0, False),
   (0.15, 2, 1.0, False),
   (0.67, 5, 1.0, False)]}

# env_flat.P[7] = {0: [(0.7, 6, 0.0, False), (0.2, 8, 1.0, True), (0.1, 1, 0, False)],
#   1: [(0.2, 6, 0.0, False), (0.7, 8, 1.0, True), (0.1, 1, 0, False)],
#   2: [(0.7, 6, 0.0, False), (0.2, 8, 1.0, True), (0.1, 1, 0, False)],
#   3: [(0.2, 6, 0.0, False), (0.7, 8, 1.0, True), (0.1, 1, 0, False)]}


nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize()

π = np.array([[0.25, 0.25, 0.25, 0.25]]*env_flat.nS)

# True Q-values
Q_values = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_values)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_values.reshape(-1))
Q_values_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_values_approx)
print()
print('RMSE:', np.sqrt(np.mean(np.square(Q_values_approx - Q_values))))
print()

True Q-values
[[1.2095  1.29496 1.29496 1.38043]
 [1.26129 1.52835 1.34935 1.6164 ]
 [0.60475 0.60475 0.69022 0.69022]
 [1.25842 1.34388 1.53381 1.61928]
 [1.30734 1.58273 1.58273 1.85813]
 [0.65367 0.65367 0.92906 0.92906]
 [0.60475 0.69022 0.60475 0.69022]
 [0.65367 0.92906 0.65367 0.92906]
 [0.      0.      0.      0.     ]]

Linear approx Q-values
[1.2095  1.29496 1.29496 1.38043 1.26129 1.52835 1.34935 1.6164  0.60475
 0.60475 0.69022 0.69022 1.25842 1.34388 1.53381 1.61928 1.30734 1.58273
 1.58273 1.85813 0.65367 0.65367 0.92906 0.92906 0.60475 0.69022 0.60475
 0.69022 0.65367 0.92906 0.65367 0.92906 0.      0.      0.      0.     ]
[[1.2095  1.29496 1.29496 1.38043]
 [1.26129 1.52835 1.34935 1.6164 ]
 [0.60475 0.60475 0.69022 0.69022]
 [1.25842 1.34388 1.53381 1.61928]
 [1.30734 1.58273 1.58273 1.85813]
 [0.65367 0.65367 0.92906 0.92906]
 [0.60475 0.69022 0.60475 0.69022]
 [0.65367 0.92906 0.65367 0.92906]
 [0.      0.      0.      0.     ]]

RMSE: 0.0



In [86]:
env_flat.P

{0: {0: [(0.67, 0, 0.0, False),
   (0.12, 3, 0.0, False),
   (0.2, 1, 0.0, False),
   (0.01, 4, 0.0, False)],
  1: [(0.16000000000000003, 0, 0.0, False),
   (0.010000000000000009, 3, 0.0, False),
   (0.6700000000000002, 1, 0.0, False),
   (0.16000000000000003, 4, 0.0, False)],
  2: [(0.16000000000000003, 0, 0.0, False),
   (0.6700000000000002, 3, 0.0, False),
   (0.010000000000000009, 1, 0.0, False),
   (0.16000000000000003, 4, 0.0, False)],
  3: [(0.010000000000000009, 0, 0.0, False),
   (0.16000000000000003, 3, 0.0, False),
   (0.16000000000000003, 1, 0.0, False),
   (0.6700000000000002, 4, 0.0, False)]},
 1: {0: [(1.0, 0, 0.18, False)],
  1: [(0.17, 0, 0.0, False),
   (0.01, 3, 0.0, False),
   (0.67, 2, 1.0, False),
   (0.15, 5, 1.0, False)],
  2: [(0.15, 0, 0.0, False),
   (0.67, 3, 0.0, False),
   (0.01, 2, 1.0, False),
   (0.17, 5, 1.0, False)],
  3: [(0.01, 0, 0.0, False),
   (0.17, 3, 0.0, False),
   (0.15, 2, 1.0, False),
   (0.67, 5, 1.0, False)]},
 2: {0: [(0.830000000000000

In [40]:
P

array([[0.67, 0.17, 0.  , 0.15, 0.01, 0.  , 0.  , 0.  , 0.  ],
       [0.16, 0.67, 0.  , 0.01, 0.16, 0.  , 0.  , 0.  , 0.  ],
       [0.16, 0.01, 0.  , 0.67, 0.16, 0.  , 0.  , 0.  , 0.  ],
       [0.01, 0.16, 0.  , 0.16, 0.67, 0.  , 0.  , 0.  , 0.  ],
       [0.67, 0.  , 0.16, 0.16, 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.16, 0.  , 0.67, 0.01, 0.  , 0.16, 0.  , 0.  , 0.  ],
       [0.16, 0.  , 0.01, 0.67, 0.  , 0.16, 0.  , 0.  , 0.  ],
       [0.01, 0.  , 0.16, 0.16, 0.  , 0.67, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.83, 0.  , 0.  , 0.17, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.83, 0.  , 0.  , 0.17, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.17, 0.  , 0.  , 0.83, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.17, 0.  , 0.  , 0.83, 0.  , 0.  , 0.  ],
       [0.67, 0.16, 0.  , 0.  , 0.  , 0.  , 0.16, 0.01, 0.  ],
       [0.16, 0.67, 0.  , 0.  , 0.  , 0.  , 0.01, 0.16, 0.  ],
       [0.16, 0.01, 0.  , 0.  , 0.  , 0.  , 0.67, 0.16, 0.  ],
       [0.01, 0.16, 0.  , 0.  , 0.  , 0.  , 0.16, 0.67,

In [19]:
Pi_w = np.zeros((nS, nS, nA))
for s in range(nS):
    Pi_w[s,s,:] = π[s]

Pi_w = Pi_w.reshape((nS, -1))

P @ Pi_w

array([[0.1675, 0.1675, 0.1675, ..., 0.    , 0.    , 0.    ],
       [0.04  , 0.04  , 0.04  , ..., 0.    , 0.    , 0.    ],
       [0.04  , 0.04  , 0.04  , ..., 0.    , 0.    , 0.    ],
       ...,
       [0.    , 0.    , 0.    , ..., 0.25  , 0.25  , 0.25  ],
       [0.    , 0.    , 0.    , ..., 0.25  , 0.25  , 0.25  ],
       [0.    , 0.    , 0.    , ..., 0.25  , 0.25  , 0.25  ]])

In [20]:
(P @ Pi_w)[0]

array([0.1675, 0.1675, 0.1675, 0.1675, 0.04  , 0.04  , 0.04  , 0.04  ,
       0.    , 0.    , 0.    , 0.    , 0.04  , 0.04  , 0.04  , 0.04  ,
       0.0025, 0.0025, 0.0025, 0.0025, 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    ])

In [47]:
p_slip = 0.2
rng = np.random.default_rng(2021)
env_flat = Env2D_Flattened(Chain2D((3,3), (2,2), p_slip=(p_slip, p_slip)))
for s in range(env_flat.nS):
    for a in range(env_flat.nA):
        futures = env_flat.P[s][a]
        probs = rng.random(len(futures))
        probs /= sum(probs)
        for idx, future in enumerate(futures):
            futures[idx] = probs[idx], *future[1:]

nS, nA = env_flat.nS, env_flat.nA
(P,R,X) = env_flat.featurize()

π = np.array([[0.25, 0.25, 0.25, 0.25]]*env_flat.nS)

# True Q-values
Q_star = algos.V2Q(env_flat, algos.policy_eval(env_flat, π, gamma), gamma)
print('True Q-values')
print(Q_star)
print()

# Linear approx Q-values
w = (np.linalg.pinv(X) @ Q_star.reshape(-1))
Q_star_approx = (X @ w).reshape((nS, nA))
print('Linear approx Q-values')
print(w)
print(Q_star_approx)
print()
print(Q_star - Q_star_approx)
print()

True Q-values
[[1.30799 1.29416 1.34612 1.34624]
 [1.46817 1.45402 1.54251 1.51711]
 [0.75313 0.72583 0.71821 0.70817]
 [1.54046 1.45725 1.52873 1.46522]
 [1.4437  1.48803 1.46553 1.80775]
 [0.82736 0.95429 0.75969 0.85848]
 [0.64983 0.65969 0.6603  0.70546]
 [0.65258 0.98298 0.67675 0.85425]
 [0.      0.      0.      0.     ]]

Linear approx Q-values
[ 1.3045  -0.00686  0.04511  1.47098 -0.01977  0.06872  0.74881 -0.01867
 -0.02629  1.53553 -0.07336 -0.00188  1.36923  0.19327  0.17078  0.83439
  0.11286 -0.08174  0.64101  0.02751  0.02812  0.6908   0.25395 -0.05228
  0.       0.      -0.     ]
[[ 1.3045   1.29764  1.34961  1.34275]
 [ 1.47098  1.4512   1.53969  1.51992]
 [ 0.74881  0.73015  0.72253  0.70386]
 [ 1.53553  1.46217  1.53365  1.46029]
 [ 1.36923  1.5625   1.54     1.73328]
 [ 0.83439  0.94725  0.75265  0.86551]
 [ 0.64101  0.66851  0.66912  0.69663]
 [ 0.6908   0.94476  0.63852  0.89248]
 [ 0.       0.      -0.       0.     ]]

[[ 0.00349 -0.00349 -0.00349  0.00349]
 [-0.0

In [48]:
P

array([[0.29006, 0.22703, 0.     , 0.36073, 0.12218, 0.     , 0.     ,
        0.     , 0.     ],
       [0.4476 , 0.18025, 0.     , 0.02539, 0.34675, 0.     , 0.     ,
        0.     , 0.     ],
       [0.10529, 0.32498, 0.     , 0.25231, 0.31742, 0.     , 0.     ,
        0.     , 0.     ],
       [0.05353, 0.61222, 0.     , 0.17035, 0.16391, 0.     , 0.     ,
        0.     , 0.     ],
       [0.27515, 0.     , 0.22086, 0.27438, 0.     , 0.22962, 0.     ,
        0.     , 0.     ],
       [0.46146, 0.     , 0.25428, 0.04298, 0.     , 0.24128, 0.     ,
        0.     , 0.     ],
       [0.18928, 0.     , 0.27799, 0.19897, 0.     , 0.33376, 0.     ,
        0.     , 0.     ],
       [0.32834, 0.     , 0.19994, 0.08933, 0.     , 0.38239, 0.     ,
        0.     , 0.     ],
       [0.     , 0.     , 0.10633, 0.     , 0.     , 0.89367, 0.     ,
        0.     , 0.     ],
       [0.     , 0.     , 0.35169, 0.     , 0.     , 0.64831, 0.     ,
        0.     , 0.     ],
       [0.     , 0. 